## Advanced R6 Usage

Complete your mastery of R6 by learning about advanced topics such as copying by reference, shared fields, cloning objects, and finalizing objects. The chapter concludes with an interview with Winston Chang, creator of the R6 package.

### Working with Environments (1)
The environment variable type is similar to a list in that it can contain other variables.

You can create a new environment using new.env().

Variables can be added to the environment using the same syntax as for lists, that is, you can use the $ and [[ operators.

In [1]:
# Define a new environment
env <- new.env()
  
# Add an element named perfect
env$perfect = c(6,28,496)

# Add an element named bases
env$bases = c("A","C","G","T")

### Working with Environments (2)
Most types of R variable use "copy by value", meaning that when you take a copy of them, the new variable has its own copy of the values. In this case, changing one variable does not affect the other.

Environments use a different system, known as "copy by reference", so that all copies are identical; changing one copy changes all the copies.

Useful biology fact: RNA bases are slightly different to DNA bases. They are A, C, G, and U.

In [2]:
# Assign lst
lst <- list(
  perfect = c(6, 28, 496),
  bases = c("A", "C", "G", "T")
)

# Copy lst
lst2 <- lst
  
# Change lst's bases element
lst$bases[4] = "U"
  
# Test lst and lst2 identical
identical(lst$bases, lst2$bases)

[1] FALSE

In [3]:
# Do the same with environments

# Copy env
env2 <- env
  
# Change env's bases element
env$bases[4] = "U"
  
# Test env and env2 identical
identical(env$bases, env2$bases)

[1] TRUE

### Static Electricity
R6 classes can use environments' copy by reference behavior to share fields between objects. To set this up, define a private field named shared. This field takes several lines to define. It should:

1. Create a new environment.
2. Assign any shared fields to that environment.
3. Return the environment.

The shared fields should be accessed via active bindings. These work in the same way as other active bindings that you have seen, but retrieve the fields using a private(dollarSing)shared(dollarSing) prefix.

In [5]:
library(R6)
# Complete the class definition
microwave_oven_factory <- R6Class(
  "MicrowaveOven",
  private = list(
    shared = {
      # Create a new environment named e
      e <- new.env()
      # Assign safety_warning into e
      e$safety_warning <- "Warning. Do not try to cook metal objects."
      # Return e
      e
    }
  ),
  active = list(
    # Add the safety_warning binding
    safety_warning = function(value) {
      if(missing(value)) {
        private$shared$safety_warning
      } else {
        private$shared$safety_warning <- value
      }
    }
  )
)

# Create two microwave ovens
a_microwave_oven <- microwave_oven_factory$new()
another_microwave_oven <- microwave_oven_factory$new()
  
# Change the safety warning for a_microwave_oven
a_microwave_oven$safety_warning <- "Warning. If the food is too hot you may scald yourself."
  
# Verify that the warning has change for another_microwave
another_microwave_oven$safety_warning

Warning message:
"package 'R6' was built under R version 3.6.3"

[1] "Warning. If the food is too hot you may scald yourself."

### Attack of the Clones
R6 objects use the same copy by reference behavior as environments. That is, if you copy an R6 object using <- assignment, then changes in one object will be reflected in the copies as well.

a_reference_copy <- an_r6_object
R6 objects have an automatically generated clone() method that is used to create a copy by value, so that changes to one copy do not affect the others.

a_value <- an_r6_object$clone()

In [6]:
library(assertive)

# define microwave_oven_factory

microwave_oven_factory <- R6Class(
  "MicrowaveOven",
  private = list(
    ..power_rating_watts = 800,
    ..power_level_watts = 800
  ),
  
  public = list(
    cook = function(time_seconds) {
      Sys.sleep(time_seconds)
      print("Your food is cooked!")
    }),
  # Add active list containing an active binding
  active = list(
    power_level_watts = function(value) {
      if(missing(value)) {
        # Return the private value
        private$..power_level_watts
      } else {
        # Assert that value is a number
        assert_is_a_number(value)
        # Assert that value is in a closed range from 0 to power rating
        assert_all_are_in_closed_range(value, 0,private$..power_rating_watts)
        # Set the private power level to value
        private$..power_level_watts <- value
      }
    }
  )
)


# Create a microwave oven
a_microwave_oven <- microwave_oven_factory$new()

# Copy a_microwave_oven using <-
assigned_microwave_oven <- a_microwave_oven
  
# Copy a_microwave_oven using clone()
cloned_microwave_oven <- a_microwave_oven$clone()
  
# Change a_microwave_oven's power level  
a_microwave_oven$power_level_watts <- 400
  
# Check a_microwave_oven & assigned_microwave_oven same 
identical(a_microwave_oven$power_level_watts, assigned_microwave_oven$power_level_watts)

# Check a_microwave_oven & cloned_microwave_oven different 
identical(a_microwave_oven$power_level_watts, cloned_microwave_oven$power_level_watts)  

Warning message:
"package 'assertive' was built under R version 3.6.3"

[1] TRUE

[1] FALSE

### This is the End (of an R6 Object)
Just as an R6 class can define a public initialize() method to run custom code when objects are created, they can also define a public finalize() method to run custom code when objects are destroyed. finalize() should take no arguments. It is typically used to close connections to databases or files, or undo side-effects such as changing global options() or graphics par()ameters.

The template for the code should be as follows.

thing_factory <- R6Class(
    
  "Thing",
  
    public = list(
    
        initialize = function(x, y, z) {
      # do something
    },
    
        finalize = function() {
      # undo something
    }

)

)

The finalize() method is called when the object is removed from memory by R's automated garbage collector. You can force a garbage collection by typing gc().

In [14]:
# install.packages("DBI")
# install.packages("RSQLite")
library(DBI)
library(RSQLite)
# From previous step
smart_microwave_oven_factory <- R6Class(
  "SmartMicrowaveOven",
  inherit = microwave_oven_factory, 
  private = list(
    conn = NULL
  ),
  public = list(
    initialize = function() {
      private$conn <- dbConnect(SQLite(), "cooking-times.sqlite")
    },
    get_cooking_time = function(food) {
      dbGetQuery(
        private$conn,
        sprintf("SELECT time_seconds FROM cooking_times WHERE food = '%s'", food)
      )
    },
    finalize = function() {
      message("Disconnecting from the cooking times database.")
      dbDisconnect(private$conn)
    }
  )
)
a_smart_microwave <- smart_microwave_oven_factory$new()

# Remove the smart microwave
rm(a_smart_microwave)  

# Force garbage collection
gc() 

also installing the dependencies 'bit', 'bit64'



package 'bit' successfully unpacked and MD5 sums checked
package 'bit64' successfully unpacked and MD5 sums checked
package 'RSQLite' successfully unpacked and MD5 sums checked

The downloaded binary packages are in
	C:\Users\Migue\AppData\Local\Temp\Rtmpg3ZrAv\downloaded_packages


Warning message:
"package 'RSQLite' was built under R version 3.6.3"

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,749850,40.1,1417382,75.7,1139135,60.9
Vcells,1316696,10.1,8388608,64.0,8199993,62.6
